In [23]:
import pandas as pd
from sqlalchemy import create_engine
import requests
import time
import os
from pandas.io.json import json_normalize

In [3]:
#START CODE HERE#
#EXTRACT#

x = ['capsules',
     'cores',
     'dragons',
     'history',
     'info',
     'landpads',
     'launches',
     'launchpads',
     'missions',
     'payloads',
     'rockets',
     'roadster',
     'ships']

collection_df = pd.DataFrame(x,columns=['collection'])
   
# params1 = {
#     'collection': x 
    
# }
# response_base = requests.get(base_url,params1).json()
# response_base
collection_df

,collection
0,capsules
1,cores
2,dragons
3,history
4,info
5,landpads
6,launches
7,launchpads
8,missions
9,payloads


In [4]:
url = 'https://api.spacexdata.com/v3/capsules'
response1 = requests.get(url).json()
response1[0]

# response1[0]['capsule_serial']
# response1[0]['capsule_id']
# response1[0]['status']
# response1[0]['original_launch']
# response1[0]['original_launch_unix']
# response1[0]['missions']
# response1[0]['landings']
# response1[0]['type']
# response1[0]['details']
# response1[0]['reuse_count']

{'capsule_serial': 'C101',
 'capsule_id': 'dragon1',
 'status': 'retired',
 'original_launch': '2010-12-08T15:43:00.000Z',
 'original_launch_unix': 1291822980,
 'missions': [{'name': 'COTS 1', 'flight': 7}],
 'landings': 0,
 'type': 'Dragon 1.0',
 'details': 'Reentered after three weeks in orbit',
 'reuse_count': 0}

In [6]:
spacex_all = []
for x in collection_df['collection']:
    url = 'https://api.spacexdata.com/v3/'+x
    response = requests.get(url).json()
    spacex_all.append(response)
#     lists.append(response)
#     response = requests.get(f'{url+row}').json()



In [7]:

collection_names = [x for x in collection_df['collection'] + '_df']

# for i in range(len(spacex_all)):
#     collection_names[i] = pd.DataFrame()
      
capsules_df = pd.DataFrame(spacex_all[0])
cores_df=pd.DataFrame(spacex_all[1])
dragons_df=pd.DataFrame(spacex_all[2])
history_df=pd.DataFrame(spacex_all[3])
Info_df=pd.DataFrame(spacex_all[4])
landpads_df=pd.DataFrame(spacex_all[5])
launches_df=pd.DataFrame(spacex_all[6])
launchpads_df=pd.DataFrame(spacex_all[7])
missions_df=pd.DataFrame(spacex_all[8])
payloads_df=pd.DataFrame(spacex_all[9])
rockets_df=pd.DataFrame(spacex_all[10])
roadster_df=pd.DataFrame(spacex_all[11])
ships_df=pd.DataFrame(spacex_all[12])

In [236]:
cores_df=pd.DataFrame(spacex_all[1])

In [70]:
capsules_df.columns

Index(['capsule_serial', 'capsule_id', 'status', 'original_launch',
       'original_launch_unix', 'missions', 'landings', 'type', 'details',
       'reuse_count'],
      dtype='object')

In [208]:
def convertdate(x):
    x = pd.to_datetime(x,format='%Y-%m-%d')
    return x

def convertunix(x):
    x = pd.to_datetime(x,unit='s')
    return x

In [149]:
capsules_df = capsules_df[['capsule_id','capsule_serial','details','original_launch','landings','missions','type','reuse_count','status']].copy()
capsules_df['original_launch'] = capsules_df['original_launch'].apply(convertdate)
capsules_df['mission_name'] = capsules_df['missions'][0:14].apply(lambda x: x[0]['name'])
capsules_df['mission_flight'] = capsules_df['missions'][0:14].apply(lambda x: x[0]['flight'])
capsules_df = capsules_df.drop(columns='missions')
#capsules df ready for import

In [187]:
capsules_df.head()

,capsule_id,capsule_serial,details,original_launch,landings,type,reuse_count,status,mission_name,mission_flight
0,dragon1,C101,Reentered after three weeks in orbit,2010-12-08 15:43:00+00:00,0,Dragon 1.0,0,retired,COTS 1,7.0
1,dragon1,C102,First Dragon spacecraft,2012-05-02 07:44:00+00:00,1,Dragon 1.0,0,retired,COTS 2,8.0
2,dragon1,C103,First of twenty missions flown under the CRS1 ...,2012-10-08 00:35:00+00:00,1,Dragon 1.0,0,unknown,CRS-1,9.0
3,dragon1,C104,None,2013-03-01 19:10:00+00:00,1,Dragon 1.0,0,unknown,CRS-2,10.0
4,dragon1,C105,First Dragon v1.1 capsule,2014-04-18 19:25:00+00:00,1,Dragon 1.1,0,unknown,CRS-3,14.0


In [135]:
capsules_missions_df = capsules_df[['capsule_serial','missions']].copy()
# capsules_missions_df = pd.concat([capsules_missions_df.drop('missions', axis=1), capsules_missions_df['missions'].apply(pd.to_dict), axis=1)
capsules_missions_df.head()

,capsule_serial,missions
0,C101,"[{'name': 'COTS 1', 'flight': 7}]"
1,C102,"[{'name': 'COTS 2', 'flight': 8}]"
2,C103,"[{'name': 'CRS-1', 'flight': 9}]"
3,C104,"[{'name': 'CRS-2', 'flight': 10}]"
4,C105,"[{'name': 'CRS-3', 'flight': 14}]"


In [12]:
# capsules_df['original_launch'] = pd.to_datetime(capsules_df['original_launch'],format='%Y-%m-%d')
# capsules_df['original_launch'] = convertdate(capsules_df['original_launch'])
# capsules_df['original_launch'] = capsules_df['original_launch'].apply(convertdate)
#three different ways to convert column original launch to datetime format.
#personally like the function convertdate as we can apply this to every dataframe that require format.


In [32]:
# capsules_df = capsules_df.drop(['missions'],axis=1)

In [11]:
# Using lambda function to separate the dictionary in missions columns and adding as new columns
# capsules_df['mission_name'] = capsules_df['missions'][0:14].apply(lambda x: x[0]['name'])
# capsules_df['mission_flight'] = capsules_df['missions'][0:14].apply(lambda x: x[0]['flight'])

In [134]:
capsules_missions_df.head()

# capsules_df = pd.concat([capsules_df['missions'].apply(pd.Series)], axis=1)
# capsules_df = pd.concat([capsules_df.drop('missions', axis=1), capsules_df['missions'].apply(pd.Series)], axis=1)

""


In [136]:
# list2 = []
# for x in capsules_missions_df['missions']:
#     list2.append(json_normalize(x))

In [65]:
# capsules_df = json_normalize(capsules_df['0'])

In [239]:
cores_df = cores_df[['core_serial','block','status','original_launch','reuse_count',\
                     'rtls_attempts','rtls_landings','asds_attempts','asds_landings','water_landing']].copy()
cores_df['block'] = cores_df['block'].fillna(value='0')
cores_df['original_launch'] = cores_df['original_launch'].apply(convertdate)
#cores_df is ready for export

In [240]:
cores_df

,core_serial,block,status,original_launch,reuse_count,rtls_attempts,rtls_landings,asds_attempts,asds_landings,water_landing
0,Merlin1A,0,lost,2006-03-24 22:30:00+00:00,0,0,0,0,0,False
1,Merlin2A,0,lost,2007-03-21 01:10:00+00:00,0,0,0,0,0,False
2,Merlin1C,0,lost,2008-08-02 03:34:00+00:00,0,0,0,0,0,False
3,Merlin2C,0,lost,2008-09-28 23:15:00+00:00,0,0,0,0,0,False
4,Merlin3C,0,lost,2009-07-13 03:35:00+00:00,0,0,0,0,0,False
...,...,...,...,...,...,...,...,...,...,...
59,B1053,5,unknown,2019-04-11 22:35:00+00:00,1,2,2,0,0,False
60,B1055,5,lost,2019-04-11 22:35:00+00:00,0,0,0,1,1,False
61,B1056,5,unknown,2019-05-04 06:48:00+00:00,2,1,1,2,2,False
62,B1057,5,lost,NaT,0,0,0,1,0,False


In [194]:
launches_df.columns

Index(['flight_number', 'mission_name', 'mission_id', 'upcoming',
       'launch_year', 'launch_date_unix', 'launch_date_utc',
       'launch_date_local', 'is_tentative', 'tentative_max_precision', 'tbd',
       'launch_window', 'rocket', 'ships', 'telemetry', 'launch_site',
       'launch_success', 'launch_failure_details', 'links', 'details',
       'static_fire_date_utc', 'static_fire_date_unix', 'timeline', 'crew',
       'last_date_update', 'last_ll_launch_date', 'last_ll_update',
       'last_wiki_launch_date', 'last_wiki_revision', 'last_wiki_update',
       'launch_date_source'],
      dtype='object')

In [203]:
launches_df['links']
#mission_patch

0      {'mission_patch': 'https://images2.imgbox.com/...
1      {'mission_patch': 'https://images2.imgbox.com/...
2      {'mission_patch': 'https://images2.imgbox.com/...
3      {'mission_patch': 'https://images2.imgbox.com/...
4      {'mission_patch': 'https://images2.imgbox.com/...
                             ...                        
99     {'mission_patch': None, 'mission_patch_small':...
100    {'mission_patch': None, 'mission_patch_small':...
101    {'mission_patch': None, 'mission_patch_small':...
102    {'mission_patch': None, 'mission_patch_small':...
103    {'mission_patch': None, 'mission_patch_small':...
Name: links, Length: 104, dtype: object

In [228]:
launches_df = launches_df[['flight_number','mission_name','mission_id','upcoming','launch_year','launch_date_unix',\
                          'is_tentative','tentative_max_precision','rocket','ships','launch_site','launch_success',\
                          'launch_failure_details','links','details','static_fire_date_utc']].copy()
launches_df['launch_date_unix'] = launches_df['launch_date_unix'].apply(convertdate)
launches_df['static_fire_date_utc'] = launches_df['static_fire_date_utc'].apply(convertdate)
launches_df.head(3)

,flight_number,mission_name,mission_id,upcoming,launch_year,launch_date_unix,is_tentative,tentative_max_precision,rocket,ships,launch_site,launch_success,launch_failure_details,links,details,static_fire_date_utc
0,1,FalconSat,[],False,2006,1970-01-01 00:00:01.143239400,False,hour,"{'rocket_id': 'falcon1', 'rocket_name': 'Falco...",[],"{'site_id': 'kwajalein_atoll', 'site_name': 'K...",False,"{'time': 33, 'altitude': None, 'reason': 'merl...",{'mission_patch': 'https://images2.imgbox.com/...,Engine failure at 33 seconds and loss of vehicle,2006-03-17 00:00:00+00:00
1,2,DemoSat,[],False,2007,1970-01-01 00:00:01.174439400,False,hour,"{'rocket_id': 'falcon1', 'rocket_name': 'Falco...",[],"{'site_id': 'kwajalein_atoll', 'site_name': 'K...",False,"{'time': 301, 'altitude': 289, 'reason': 'harm...",{'mission_patch': 'https://images2.imgbox.com/...,Successful first stage burn and transition to ...,NaT
2,3,Trailblazer,[],False,2008,1970-01-01 00:00:01.217648040,False,hour,"{'rocket_id': 'falcon1', 'rocket_name': 'Falco...",[],"{'site_id': 'kwajalein_atoll', 'site_name': 'K...",False,"{'time': 140, 'altitude': 35, 'reason': 'resid...",{'mission_patch': 'https://images2.imgbox.com/...,Residual stage 1 thrust led to collision betwe...,NaT


In [227]:
# launches_rocket_df = pd.DataFrame(json_normalize(launches_df['rocket']))
# launches_rocket_df['first_'] = launches_rocket_df['missions'][0:14].apply(lambda x: x[0]['name'])
# launches_rocket_df['mission_flight'] = launches_rocket_df['missions'][0:14].apply(lambda x: x[0]['flight'])
#work on this later

In [13]:
# rearranging the columns and exporting to csv
# capsules_df = capsules_df[['capsule_id','capsule_serial','details','original_launch','landings','mission_name','mission_flight','type','reuse_count','status']].copy()

# capsules_df.to_csv('capsules.csv')

In [66]:

# pd.concat([df.drop(['b'], axis=1), df['b'].apply(pd.Series)], axis=1)

In [16]:
connection_string = 'postgres:root123@localhost:5432/mission_spacex'
engine = create_engine(f'postgresql://{connection_string}')

In [17]:
engine.table_names()

['capsules',
 'capsule_mission',
 'cores',
 'cores_mission',
 'dragons',
 'thrusters',
 'history']

In [36]:
capsules_df.to_sql(name='capsules', con=engine,if_exists='append', index=False)
cores_df.to_sql(name='cores',con=engine,if_exists='append',index=False)


In [226]:
launches_df.to_sql(name='launches',con=engine,if_exists='append',index=False)

ProgrammingError: (psycopg2.ProgrammingError) can't adapt type 'dict'
[SQL: INSERT INTO launches (flight_number, mission_name, mission_id, upcoming, launch_year, launch_date_unix, is_tentative, tentative_max_precision, rocket, ships, launch_site, launch_success, launch_failure_details, links, details, static_fire_date_utc) VALUES (%(flight_number)s, %(mission_name)s, %(mission_id)s, %(upcoming)s, %(launch_year)s, %(launch_date_unix)s, %(is_tentative)s, %(tentative_max_precision)s, %(rocket)s, %(ships)s, %(launch_site)s, %(launch_success)s, %(launch_failure_details)s, %(links)s, %(details)s, %(static_fire_date_utc)s)]
[parameters: ({'flight_number': 1, 'mission_name': 'FalconSat', 'mission_id': [], 'upcoming': False, 'launch_year': '2006', 'launch_date_unix': datetime.datetime(1970, 1, 1, 0, 0, 1, 143239), 'is_tentative': False, 'tentative_max_precision': 'hour', 'rocket': {'rocket_id': 'falcon1', 'rocket_name': 'Falcon 1', 'rocket_type': 'Merlin A', 'first_stage': {'cores': [{'core_ser